<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Libraries-And-Settings" data-toc-modified-id="Libraries-And-Settings-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Libraries And Settings</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Additional-Cleanups" data-toc-modified-id="Additional-Cleanups-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Additional Cleanups</a></span></li></ul></li><li><span><a href="#Applying-Random-Forest" data-toc-modified-id="Applying-Random-Forest-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Applying Random Forest</a></span><ul class="toc-item"><li><span><a href="#Applying-SMOTE" data-toc-modified-id="Applying-SMOTE-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Applying SMOTE</a></span></li><li><span><a href="#Run-RandomForest-Classifier-with-510-Estimators" data-toc-modified-id="Run-RandomForest-Classifier-with-510-Estimators-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Run RandomForest Classifier with 510 Estimators</a></span></li><li><span><a href="#Hyperparameters-and-GridSearch" data-toc-modified-id="Hyperparameters-and-GridSearch-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Hyperparameters and GridSearch</a></span></li></ul></li></ul></div>

## Libraries And Settings

In [1]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px 

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from scipy.stats import norm
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from IPython.core.display import HTML

In [2]:
# Set Settings

# Matplotlib
plt.style.use('ggplot')
%matplotlib inline

# Pandas
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)

## Dataset

In [3]:
# Read in dataset with derates for training
data = pd.read_csv('../data/train_codes_4hr.csv', low_memory=False)

# Check current state
display(data.shape)
display(data.head())
display(data.info())

(443229, 512)

,Unnamed: 0,active_transition_count,accelerator_pedal,barometric_pressure,cruise_control_active,cruise_control_set_speed,distance_ltd,engine_coolant_temperature,engine_load,engine_oil_pressure,engine_oil_temperature,engine_rpm,engine_time_ltd,fuel_level,fuel_ltd,fuel_rate,fuel_temperature,ign_status,intake_manifold_temperature,parking_brake,speed,switched_battery_voltage,throttle,turbo_boost_pressure,target,spn_0,spn_100,spn_101,spn_102,spn_1023,spn_1024,spn_1028,spn_103,spn_1043,spn_1045,spn_105,spn_1056,spn_1059,spn_1067,spn_1068,spn_107,spn_1071,spn_1072,spn_1075,spn_1078,spn_108,spn_1081,spn_110,spn_111,spn_1127,spn_114863,spn_116,spn_1172,spn_1176,spn_118,spn_1209,spn_1213,spn_1231,spn_1235,spn_1236,spn_1239,spn_1247,spn_125,spn_127,spn_1321,spn_1322,spn_1323,spn_1324,spn_1325,spn_1326,spn_1327,spn_1328,spn_1347,spn_1349,spn_13600,spn_139296,spn_1464,spn_1481,spn_1482,spn_1483,spn_1487,spn_153,spn_153931,spn_1569,spn_157,spn_158,spn_16,spn_160,spn_1612,spn_1659,spn_1668,spn_167,spn_1675,spn_168,spn_17096,spn_171,spn_173,spn_174,spn_175,spn_17590,spn_1761,spn_177,spn_1787,spn_1807,spn_1808,spn_1809,spn_1815,spn_184,spn_188,spn_190,spn_191,spn_196608,spn_2000,spn_2017,spn_2023,spn_2029,spn_228,spn_235,spn_236,spn_237,spn_245,spn_247,spn_248,spn_251,spn_252,spn_255,spn_256,spn_25780,spn_2579,spn_2623,spn_2629,spn_2630,spn_2659,spn_27,spn_2791,spn_2795,spn_2863,spn_2866,spn_2912,spn_2917,spn_29902,spn_3031,spn_3058,spn_3060,spn_3064,spn_32000,spn_3216,spn_3217,spn_3218,spn_3222,spn_3226,spn_3227,spn_3228,spn_3241,spn_3242,spn_3245,spn_3246,spn_3249,spn_3251,spn_3253,spn_33,spn_335040,spn_3360,spn_3361,spn_3362,spn_3363,spn_3364,spn_3464,spn_3480,spn_3482,spn_3490,spn_3509,spn_3510,spn_3511,spn_3512,spn_3513,spn_3515,spn_3521,spn_35527,spn_3556,spn_3583,spn_3597,spn_36017,spn_3605,spn_3610,spn_3663,spn_3695,spn_3696,spn_3697,spn_3698,spn_37,spn_3703,spn_3720,spn_38,spn_3821,spn_39093,spn_3936,spn_4094,spn_4095,spn_4096,spn_411,spn_412,spn_4219,spn_4220,spn_4276,spn_43088,spn_4331,spn_4334,spn_4339,spn_4340,spn_4342,spn_4344,spn_4346,spn_4349,spn_4354,spn_4356,spn_4360,spn_4363,spn_4364,spn_4375,spn_4376,spn_4380,spn_4382,spn_441,spn_442,spn_444,spn_47284,spn_4752,spn_4765,spn_4766,spn_4792,spn_4794,spn_4795,spn_4796,spn_4811,spn_4812,spn_4813,spn_5018,spn_5019,spn_5024,spn_5031,spn_50353,spn_5052,spn_51,spn_5109,spn_5110,spn_5111,spn_5112,spn_5113,spn_5114,...,spn_520413,spn_520953,spn_521032,spn_522,spn_523530,spn_523531,spn_523543,spn_524033,spn_524037,spn_524071,spn_524287,spn_5245,spn_525,spn_5298,spn_5319,spn_5321,spn_5357,spn_5392,spn_5394,spn_5395,spn_5396,spn_5397,spn_5442,spn_5443,spn_5444,spn_5485,spn_5491,spn_5569,spn_5571,spn_5579,spn_558,spn_5585,spn_560,spn_5614,spn_5615,spn_5616,spn_5625,spn_563,spn_5742,spn_5743,spn_5745,spn_5746,spn_576,spn_577,spn_578,spn_583,spn_5835,spn_5848,spn_5851,spn_5853,spn_5862,spn_5902,spn_5903,spn_5909,spn_5939,spn_5941,spn_5942,spn_5953,spn_596,spn_603,spn_609,spn_611,spn_612,spn_614,spn_6145,spn_6146,spn_6147,spn_6148,spn_624,spn_627,spn_628,spn_629,spn_630,spn_632,spn_633,spn_636,spn_639,spn_641,spn_647,spn_649,spn_651,spn_652,spn_653,spn_65302,spn_65303,spn_654,spn_655,spn_65535,spn_656,spn_6713,spn_6773,spn_6780,spn_6802,spn_70,spn_705,spn_709,spn_723,spn_729,spn_7321,spn_7323,spn_74,spn_75,spn_751,spn_752,spn_76339,spn_767,spn_768,spn_77,spn_78,spn_781,spn_78132,spn_7827,spn_7847,spn_7854,spn_788,spn_789,spn_790,spn_791,spn_792,spn_793,spn_794,spn_795,spn_796,spn_797,spn_798,spn_799,spn_800,spn_801,spn_802,spn_803,spn_805,spn_806,spn_807,spn_81,spn_810,spn_829,spn_830,spn_84,spn_862,spn_88121,spn_886,spn_904,spn_905,spn_906,spn_907,spn_91,spn_917,spn_92,spn_929,spn_9295,spn_932,spn_933,spn_934,spn_937,spn_938,spn_939,spn_94,spn_940,spn_941,spn_95,spn_96,spn_97,spn_976,spn_98,fmi_0,fmi_1,fmi_2,fmi_3,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9,fmi_10,fmi_11,fmi_12,fmi_13,fmi_14,fmi_15,fmi_16,fmi_17,fmi_18,fmi_19,fmi_20,fmi_21,fmi_22,fmi_23,fmi_29,fmi_31,lamp_status_0,lamp_status_2,lamp

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443229 entries, 0 to 443228
Columns: 512 entries, Unnamed: 0 to ecu_model_unknown
dtypes: bool(3), float64(19), int64(490)
memory usage: 1.7 GB


None

### Additional Cleanups

In [4]:
# Drop unneeded columns
data = data.drop(columns=['Unnamed: 0'])

# Fill out NAs with Mean
column_means = data.mean()
data = data.fillna(column_means)

# Check final state
display(data.shape)
display(data.head())
display(data.info())

(443229, 511)

,active_transition_count,accelerator_pedal,barometric_pressure,cruise_control_active,cruise_control_set_speed,distance_ltd,engine_coolant_temperature,engine_load,engine_oil_pressure,engine_oil_temperature,engine_rpm,engine_time_ltd,fuel_level,fuel_ltd,fuel_rate,fuel_temperature,ign_status,intake_manifold_temperature,parking_brake,speed,switched_battery_voltage,throttle,turbo_boost_pressure,target,spn_0,spn_100,spn_101,spn_102,spn_1023,spn_1024,spn_1028,spn_103,spn_1043,spn_1045,spn_105,spn_1056,spn_1059,spn_1067,spn_1068,spn_107,spn_1071,spn_1072,spn_1075,spn_1078,spn_108,spn_1081,spn_110,spn_111,spn_1127,spn_114863,spn_116,spn_1172,spn_1176,spn_118,spn_1209,spn_1213,spn_1231,spn_1235,spn_1236,spn_1239,spn_1247,spn_125,spn_127,spn_1321,spn_1322,spn_1323,spn_1324,spn_1325,spn_1326,spn_1327,spn_1328,spn_1347,spn_1349,spn_13600,spn_139296,spn_1464,spn_1481,spn_1482,spn_1483,spn_1487,spn_153,spn_153931,spn_1569,spn_157,spn_158,spn_16,spn_160,spn_1612,spn_1659,spn_1668,spn_167,spn_1675,spn_168,spn_17096,spn_171,spn_173,spn_174,spn_175,spn_17590,spn_1761,spn_177,spn_1787,spn_1807,spn_1808,spn_1809,spn_1815,spn_184,spn_188,spn_190,spn_191,spn_196608,spn_2000,spn_2017,spn_2023,spn_2029,spn_228,spn_235,spn_236,spn_237,spn_245,spn_247,spn_248,spn_251,spn_252,spn_255,spn_256,spn_25780,spn_2579,spn_2623,spn_2629,spn_2630,spn_2659,spn_27,spn_2791,spn_2795,spn_2863,spn_2866,spn_2912,spn_2917,spn_29902,spn_3031,spn_3058,spn_3060,spn_3064,spn_32000,spn_3216,spn_3217,spn_3218,spn_3222,spn_3226,spn_3227,spn_3228,spn_3241,spn_3242,spn_3245,spn_3246,spn_3249,spn_3251,spn_3253,spn_33,spn_335040,spn_3360,spn_3361,spn_3362,spn_3363,spn_3364,spn_3464,spn_3480,spn_3482,spn_3490,spn_3509,spn_3510,spn_3511,spn_3512,spn_3513,spn_3515,spn_3521,spn_35527,spn_3556,spn_3583,spn_3597,spn_36017,spn_3605,spn_3610,spn_3663,spn_3695,spn_3696,spn_3697,spn_3698,spn_37,spn_3703,spn_3720,spn_38,spn_3821,spn_39093,spn_3936,spn_4094,spn_4095,spn_4096,spn_411,spn_412,spn_4219,spn_4220,spn_4276,spn_43088,spn_4331,spn_4334,spn_4339,spn_4340,spn_4342,spn_4344,spn_4346,spn_4349,spn_4354,spn_4356,spn_4360,spn_4363,spn_4364,spn_4375,spn_4376,spn_4380,spn_4382,spn_441,spn_442,spn_444,spn_47284,spn_4752,spn_4765,spn_4766,spn_4792,spn_4794,spn_4795,spn_4796,spn_4811,spn_4812,spn_4813,spn_5018,spn_5019,spn_5024,spn_5031,spn_50353,spn_5052,spn_51,spn_5109,spn_5110,spn_5111,spn_5112,spn_5113,spn_5114,spn_5115,...,spn_520413,spn_520953,spn_521032,spn_522,spn_523530,spn_523531,spn_523543,spn_524033,spn_524037,spn_524071,spn_524287,spn_5245,spn_525,spn_5298,spn_5319,spn_5321,spn_5357,spn_5392,spn_5394,spn_5395,spn_5396,spn_5397,spn_5442,spn_5443,spn_5444,spn_5485,spn_5491,spn_5569,spn_5571,spn_5579,spn_558,spn_5585,spn_560,spn_5614,spn_5615,spn_5616,spn_5625,spn_563,spn_5742,spn_5743,spn_5745,spn_5746,spn_576,spn_577,spn_578,spn_583,spn_5835,spn_5848,spn_5851,spn_5853,spn_5862,spn_5902,spn_5903,spn_5909,spn_5939,spn_5941,spn_5942,spn_5953,spn_596,spn_603,spn_609,spn_611,spn_612,spn_614,spn_6145,spn_6146,spn_6147,spn_6148,spn_624,spn_627,spn_628,spn_629,spn_630,spn_632,spn_633,spn_636,spn_639,spn_641,spn_647,spn_649,spn_651,spn_652,spn_653,spn_65302,spn_65303,spn_654,spn_655,spn_65535,spn_656,spn_6713,spn_6773,spn_6780,spn_6802,spn_70,spn_705,spn_709,spn_723,spn_729,spn_7321,spn_7323,spn_74,spn_75,spn_751,spn_752,spn_76339,spn_767,spn_768,spn_77,spn_78,spn_781,spn_78132,spn_7827,spn_7847,spn_7854,spn_788,spn_789,spn_790,spn_791,spn_792,spn_793,spn_794,spn_795,spn_796,spn_797,spn_798,spn_799,spn_800,spn_801,spn_802,spn_803,spn_805,spn_806,spn_807,spn_81,spn_810,spn_829,spn_830,spn_84,spn_862,spn_88121,spn_886,spn_904,spn_905,spn_906,spn_907,spn_91,spn_917,spn_92,spn_929,spn_9295,spn_932,spn_933,spn_934,spn_937,spn_938,spn_939,spn_94,spn_940,spn_941,spn_95,spn_96,spn_97,spn_976,spn_98,fmi_0,fmi_1,fmi_2,fmi_3,fmi_4,fmi_5,fmi_6,fmi_7,fmi_8,fmi_9,fmi_10,fmi_11,fmi_12,fmi_13,fmi_14,fmi_15,fmi_16,fmi_17,fmi_18,fmi_19,fmi_20,fmi_21,fmi_22,fmi_23,fmi_29,fmi_31,lamp_status_0,lamp_status_2,lamp_s

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443229 entries, 0 to 443228
Columns: 511 entries, active_transition_count to ecu_model_unknown
dtypes: bool(3), float64(19), int64(489)
memory usage: 1.7 GB


None

## Applying Random Forest

In [5]:
# Define predictors
X = data.drop('target', axis=1) # Pandas Dataframe as 2D Array: Array of Arrays

# Define target
y = data['target'] # 1D Array

In [6]:
# Train-Test Spit
# TODO: Replace with CV
# Question: If we can only apply SMOTE on the training set, how can we do CV with it?
X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    random_state=321, 
    stratify=y
)

### Applying SMOTE

In [7]:
# Import libraries
from imblearn.over_sampling import SMOTE

In [8]:
# Create a SMOTE oversampler
oversampler = SMOTE(
    k_neighbors=5, 
    random_state=321,
    #n_jobs=-1
)

In [9]:
# Fit and resample
X_smote, y_smote = oversampler.fit_resample(X_train, y_train)

In [10]:
# Check result
y_smote.value_counts()# Check result
y_smote.value_counts()

0    332111
1    332111
Name: target, dtype: int64

### Run RandomForest Classifier with 510 Estimators

In [11]:
# Create a Random Forest Classifier and Fit on it
rf = RandomForestClassifier(
    n_estimators=510, 
    random_state=42
).fit(X_smote, y_smote)

In [12]:
# Predict and measure
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9991426611796982

In [13]:
# Confusing Matrix
confusion_matrix(y_test, y_pred)

array([[110699,      5],
       [    90,     14]], dtype=int64)

- True Negative Derates: 110699
- True Positive Derates: 14
- False Negative Derates: 90
- False Positive Derates: 5

In [14]:
# Check metrics
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    110704
           1       0.74      0.13      0.23       104

    accuracy                           1.00    110808
   macro avg       0.87      0.57      0.61    110808
weighted avg       1.00      1.00      1.00    110808



In [15]:
# Check Feature Importance
feature_importances = pd.DataFrame({
    'feature': X.columns, 
    'importance': rf.feature_importances_
})
# Check the top 10 features
feature_importances.sort_values('importance', ascending = False).head(10)

,feature,importance
0,active_transition_count,0.082797
458,lamp_status_1279,0.050844
457,lamp_status_1023,0.048419
4,cruise_control_set_speed,0.044796
5,distance_ltd,0.037035
46,spn_111,0.036848
13,fuel_ltd,0.035106
11,engine_time_ltd,0.034951
441,fmi_17,0.030066
468,lamp_status_17407,0.028137


### Hyperparameters and GridSearch

This is some heavy-duty stuff here